In [3]:
import pandas as pd
from postal.expand import expand_address
import numpy as np 
import matplotlib.pyplot as plt

IMPORT DES DONNEES

In [44]:
df = pd.read_csv(
    "/Users/noelinecasteil/Documents/statapp/DPE/DPE_ADEME/dpe-v2-logements-existants.csv",
    sep=",",  # Séparateur CSV
    encoding="utf-8",
    low_memory=False)

df['Date_réception_DPE'].count()

4537525

In [3]:
s3 = s3_connection()

NameError: name 's3_connection' is not defined

In [4]:
path_logements_existants = "clichere/diffusion/DPE/DPE_ADEME/dpe-v2-logements-existants.csv"
df = s3.read_file_from_s3(path_logements_existants)

/home/onyxia/STATAPP/helpers.py:35: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(file_in, usecols=columns_to_select, dtype=dtype_spec, sep=sep)


In [57]:
df2 = pd.read_csv(
    "/Users/noelinecasteil/Documents/statapp/DPE/DPE_ADEME/dpe-v2-logements-neufs.csv",
    sep=",",  # Séparateur CSV
    encoding="utf-8",
    low_memory=False)

df2['Date_réception_DPE'].count()

537952

In [5]:
path_logements_existants = "clichere/diffusion/DPE/DPE_ADEME/dpe-v2-logements-existants.csv"
df2 = s3.read_file_from_s3(path_logements_existants)

/home/onyxia/STATAPP/helpers.py:35: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(file_in, usecols=columns_to_select, dtype=dtype_spec, sep=sep)


In [15]:
vf = pd.read_csv(
    "/Users/noelinecasteil/Documents/statapp/ValeursFoncieres/valeursfoncieres-2022.txt",
    sep="|",  
    encoding="utf-8",
    low_memory=False
)

In [6]:
path_valeursfoncieres_2022 = "clichere/diffusion/Valeursfoncières/valeursfoncieres-2022.txt"
vf = s3.read_file_from_s3(path_valeursfoncieres_2022, sep="|")

/home/onyxia/STATAPP/helpers.py:35: DtypeWarning: Columns (18,23,24,26,28,29,31,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_in, usecols=columns_to_select, dtype=dtype_spec, sep=sep)


In [16]:
# Convertir explicitement toutes les colonnes en chaînes
vf['Adresse'] = vf['No voie'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
                vf['Type de voie'].fillna('').astype(str) + " " + \
                vf['Voie'].fillna('').astype(str) + ", " + \
                vf['Code postal'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
                vf['Commune'].fillna('').astype(str)

vf['Adresse'] = vf['Adresse'].str.strip().replace(r'^\s*$', None, regex=True)  # Supprime les adresses vides

In [8]:
def normalize_address(address):
    if pd.isna(address) or address.strip() == '':
        return None  
    try:
        normalized = expand_address(address)  
        return normalized[0] if normalized else None  # Ne garde que la première version
    except Exception as e:
        print(f"Erreur avec l'adresse '{address}': {e}")
        return None

In [23]:
df['Date_réception_DPE'] = pd.to_datetime(df['Date_réception_DPE'], errors='coerce')
df = df[df['Date_réception_DPE'].dt.year == 2022].copy()

In [24]:
vf = vf[vf['Code postal'].notna()]
df = df[df['N°_département_(BAN)'].notna()]

In [25]:
vf['Code postal'] = vf['Code postal'].astype(int).astype(str)
df.loc[:, 'N°_département_(BAN)'] = df['N°_département_(BAN)'].astype(str)

In [17]:
import re

def normalize_vf_address(address, code_postal):
    """ Normalise l'adresse et supprime l'arrondissement après 'PARIS' uniquement si le département est 75. """
    normalized_address = normalize_address(address)  

    if pd.notna(normalized_address) and str(code_postal).startswith("75"):
        # Supprime le numéro après "PARIS" uniquement pour le département 75
        normalized_address = re.sub(r'(paris) \d{2}$', r'\1', normalized_address, flags=re.IGNORECASE)
    
    return normalized_address


## TEST AVEC 5% POUR LES SURFACES

In [21]:
def test_match2(vf, df):
    merged = []


    #boucle par département
    for department in df['N°_département_(BAN)'].unique():  
        print(f"Traitement du département : {department}")
        
        #filtre
        vf_dept = vf[vf['Code postal'].astype(str).str.startswith(department)].copy()
        df_dept = df[df['N°_département_(BAN)']==department].copy()

        #normalisation adresses
        vf_dept['Adresse'] = vf_dept['Adresse'].str.strip().str.replace(r'\s+', ' ', regex=True)
        vf_dept['Adresse_Normalisee'] = vf_dept.apply(lambda row: normalize_vf_address(row['Adresse'], row['Code postal']), axis=1)
        df_dept['Adresse_Normalisee'] = df_dept['Adresse_(BAN)'].apply(normalize_address)

        adresse_counts = vf_dept['Adresse_Normalisee'].dropna().value_counts()
        unique1 = list(adresse_counts[adresse_counts == 1].index)
        doublons1 = list(adresse_counts[adresse_counts > 1].index)
        final = unique1 + doublons1
        set_final = set(final)

        adresse_counts2 = df_dept['Adresse_Normalisee'].dropna().value_counts()
        unique2 = list(adresse_counts2[adresse_counts2 == 1].index)
        doublons2 = list(adresse_counts2[adresse_counts2 > 1].index)
        final2 = unique2 + doublons2
        set_final2 = set(final2)

        commun = set_final.intersection(set_final2)

        vf_dept['Surface Carrez du 1er lot'] = pd.to_numeric(
            vf_dept['Surface Carrez du 1er lot'].astype(str).str.replace(',', '.'), errors='coerce'
        )
        df_dept['Surface_habitable_logement'] = pd.to_numeric(
            df_dept['Surface_habitable_logement'].astype(str).str.replace(',', '.'), errors='coerce'
        )

        for adresse in commun:
            dfsub = df_dept[df_dept['Adresse_Normalisee'] == adresse]
            vfsub = vf_dept[vf_dept['Adresse_Normalisee'] == adresse]

            for _, row2 in dfsub.iterrows():
                best_match = None
                best_value = -1

                for _, row1 in vfsub.iterrows():
                    surface1 = row1['Surface Carrez du 1er lot']
                    surface2 = row2['Surface_habitable_logement']

                    #si surfaces identiques on match direct
                    if surface1 == surface2:
                        best_match = row1
                        break

                    #ecart inf à 5%
                    if abs(surface1 - surface2) / max(surface1, surface2) < 0.05:
                        valeur_fonciere = pd.to_numeric(str(row1.get('Valeur fonciere', 0)).replace(',', '.'), errors='coerce')
                        if valeur_fonciere > best_value:
                            best_value = valeur_fonciere
                            best_match = row1

                if best_match is not None:
                    merged.append({**row2.to_dict(), **best_match.to_dict()})

    #df des résultats fusionnés
    df = pd.DataFrame(merged)
    return df


In [ ]:

#application fonction
test2 = test_match2(vf, df)

Traitement du département : 13
Traitement du département : 92
Traitement du département : 69
Traitement du département : 03
Traitement du département : 56
Traitement du département : 75
Traitement du département : 29
Traitement du département : 91
Traitement du département : 62
Traitement du département : 21
Traitement du département : 11
Traitement du département : 02
Traitement du département : 59
Traitement du département : 94
Traitement du département : 44
Traitement du département : 50
Traitement du département : 37
Traitement du département : 81
Traitement du département : 67
Traitement du département : 93
Traitement du département : 83
Traitement du département : 63
Traitement du département : 95
Traitement du département : 76
Traitement du département : 33
Traitement du département : 78
Traitement du département : 38
Traitement du département : 51
Traitement du département : 57
Traitement du département : 77
Traitement du département : 35
Traitement du département : 54
Traiteme

In [88]:
test2.to_csv("testfinal.csv", index=False)

In [14]:
print(len(vf))

3764712


In [15]:
print(len(df))

2405317


In [16]:
print(len(test2))

70665


### Etudes doublons 

In [17]:
df_final = pd.read_csv("testfinal.csv")

# Identifier les doublons (conservés malgré le drop dans le script précédent ?)
doublons = df_final[df_final.duplicated(
    subset=['Adresse_Normalisee', 'Surface_habitable_logement', 'Valeur fonciere'], 
    keep=False
)]


/tmp/ipykernel_179738/2613097225.py:1: DtypeWarning: Columns (15,58,60,62,65,67,75) have mixed types. Specify dtype option on import or set low_memory=False.
  df_final = pd.read_csv("testfinal.csv")


In [26]:
from IPython.display import display

#display(doublons.sort_values(by='Adresse_Normalisee').head(100))


In [90]:
result_df = test2.drop_duplicates(subset=['Date_réception_DPE', 'Adresse_Normalisee', 'Surface_habitable_logement', 'Valeur fonciere', 'N°_étage_appartement', 'Etiquette_DPE', "Complément_d'adresse_logement", "Complément_d'adresse_bâtiment", "Date_établissement_DPE"])

In [91]:
result_df.to_csv("result5%.csv", index=False)

In [92]:
print(len(result_df))

77148


In [47]:
df_final = pd.read_csv("testfinal.csv")

# Identifier les doublons (conservés malgré le drop dans le script précédent ?)
doublons1 = df_final[df_final.duplicated(
    subset=['Date_réception_DPE', 'Adresse_Normalisee', 'Surface_habitable_logement', 'Valeur fonciere', 'N°_étage_appartement', 'Etiquette_DPE', "Complément_d'adresse_logement"], 
    keep=False
)]

/tmp/ipykernel_179738/3911531778.py:1: DtypeWarning: Columns (15,58,60,62,65,67,75) have mixed types. Specify dtype option on import or set low_memory=False.
  df_final = pd.read_csv("testfinal.csv")


In [25]:
from IPython.display import display
pd.set_option('display.max_columns', None)

#display(doublons1.sort_values(by='Adresse_Normalisee').head(100))

## TEST AVEC 10% POUR LES SURFACES

In [94]:
def test_match3(vf, df):
    merged = []


    #boucle par département
    for department in df['N°_département_(BAN)'].unique():  
        print(f"Traitement du département : {department}")
        
        #filtre
        vf_dept = vf[vf['Code postal'].astype(str).str.startswith(department)].copy()
        df_dept = df[df['N°_département_(BAN)']==department].copy()

        #normalisation adresses
        vf_dept['Adresse'] = vf_dept['Adresse'].str.strip().str.replace(r'\s+', ' ', regex=True)
        vf_dept['Adresse_Normalisee'] = vf_dept.apply(lambda row: normalize_vf_address(row['Adresse'], row['Code postal']), axis=1)
        df_dept['Adresse_Normalisee'] = df_dept['Adresse_(BAN)'].apply(normalize_address)

        adresse_counts = vf_dept['Adresse_Normalisee'].dropna().value_counts()
        unique1 = list(adresse_counts[adresse_counts == 1].index)
        doublons1 = list(adresse_counts[adresse_counts > 1].index)
        final = unique1 + doublons1
        set_final = set(final)

        adresse_counts2 = df_dept['Adresse_Normalisee'].dropna().value_counts()
        unique2 = list(adresse_counts2[adresse_counts2 == 1].index)
        doublons2 = list(adresse_counts2[adresse_counts2 > 1].index)
        final2 = unique2 + doublons2
        set_final2 = set(final2)

        commun = set_final.intersection(set_final2)

        vf_dept['Surface Carrez du 1er lot'] = pd.to_numeric(
            vf_dept['Surface Carrez du 1er lot'].astype(str).str.replace(',', '.'), errors='coerce'
        )
        df_dept['Surface_habitable_logement'] = pd.to_numeric(
            df_dept['Surface_habitable_logement'].astype(str).str.replace(',', '.'), errors='coerce'
        )

        for adresse in commun:
            dfsub = df_dept[df_dept['Adresse_Normalisee'] == adresse]
            vfsub = vf_dept[vf_dept['Adresse_Normalisee'] == adresse]

            for _, row2 in dfsub.iterrows():
                best_match = None
                best_value = -1

                for _, row1 in vfsub.iterrows():
                    surface1 = row1['Surface Carrez du 1er lot']
                    surface2 = row2['Surface_habitable_logement']

                    #si surfaces identiques on match direct
                    if surface1 == surface2:
                        best_match = row1
                        break

                    #ecart inf à 10%
                    if abs(surface1 - surface2) / max(surface1, surface2) < 0.1:
                        valeur_fonciere = pd.to_numeric(str(row1.get('Valeur fonciere', 0)).replace(',', '.'), errors='coerce')
                        if valeur_fonciere > best_value:
                            best_value = valeur_fonciere
                            best_match = row1

                if best_match is not None:
                    merged.append({**row2.to_dict(), **best_match.to_dict()})

    #df des résultats fusionnés
    df = pd.DataFrame(merged)
    return df

#application fonction
test3 = test_match3(vf, df)

Traitement du département : 13
Traitement du département : 92
Traitement du département : 69
Traitement du département : 03
Traitement du département : 56
Traitement du département : 75
Traitement du département : 29
Traitement du département : 91
Traitement du département : 62
Traitement du département : 21
Traitement du département : 11
Traitement du département : 02
Traitement du département : 59
Traitement du département : 94
Traitement du département : 44
Traitement du département : 50
Traitement du département : 37
Traitement du département : 81
Traitement du département : 67
Traitement du département : 93
Traitement du département : 83
Traitement du département : 63
Traitement du département : 95
Traitement du département : 76
Traitement du département : 33
Traitement du département : 78
Traitement du département : 38
Traitement du département : 51
Traitement du département : 57
Traitement du département : 77
Traitement du département : 35
Traitement du département : 54
Traiteme

In [95]:
print(len(test3))

94314


In [96]:
result10 = test3.drop_duplicates(subset=['Date_réception_DPE', 'Adresse_Normalisee', 'Surface_habitable_logement', 'Valeur fonciere', 'N°_étage_appartement', 'Etiquette_DPE', "Complément_d'adresse_logement", "Complément_d'adresse_bâtiment", "Date_établissement_DPE"])

In [97]:
print(len(result10))

88831


In [98]:
result10.to_csv("result10%.csv", index=False)

## TEST SEUIL DE 3%

In [27]:
def test_match4(vf, df):
    merged = []


    #boucle par département
    for department in df['N°_département_(BAN)'].unique():  
        print(f"Traitement du département : {department}")
        
        #filtre
        vf_dept = vf[vf['Code postal'].astype(str).str.startswith(department)].copy()
        df_dept = df[df['N°_département_(BAN)']==department].copy()

        #normalisation adresses
        vf_dept['Adresse'] = vf_dept['Adresse'].str.strip().str.replace(r'\s+', ' ', regex=True)
        vf_dept['Adresse_Normalisee'] = vf_dept.apply(lambda row: normalize_vf_address(row['Adresse'], row['Code postal']), axis=1)
        df_dept['Adresse_Normalisee'] = df_dept['Adresse_(BAN)'].apply(normalize_address)

        adresse_counts = vf_dept['Adresse_Normalisee'].dropna().value_counts()
        unique1 = list(adresse_counts[adresse_counts == 1].index)
        doublons1 = list(adresse_counts[adresse_counts > 1].index)
        final = unique1 + doublons1
        set_final = set(final)

        adresse_counts2 = df_dept['Adresse_Normalisee'].dropna().value_counts()
        unique2 = list(adresse_counts2[adresse_counts2 == 1].index)
        doublons2 = list(adresse_counts2[adresse_counts2 > 1].index)
        final2 = unique2 + doublons2
        set_final2 = set(final2)

        commun = set_final.intersection(set_final2)

        vf_dept['Surface Carrez du 1er lot'] = pd.to_numeric(
            vf_dept['Surface Carrez du 1er lot'].astype(str).str.replace(',', '.'), errors='coerce'
        )
        df_dept['Surface_habitable_logement'] = pd.to_numeric(
            df_dept['Surface_habitable_logement'].astype(str).str.replace(',', '.'), errors='coerce'
        )

        for adresse in commun:
            dfsub = df_dept[df_dept['Adresse_Normalisee'] == adresse]
            vfsub = vf_dept[vf_dept['Adresse_Normalisee'] == adresse]

            for _, row2 in dfsub.iterrows():
                best_match = None
                best_value = -1

                for _, row1 in vfsub.iterrows():
                    surface1 = row1['Surface Carrez du 1er lot']
                    surface2 = row2['Surface_habitable_logement']

                    #si surfaces identiques on match direct
                    if surface1 == surface2:
                        best_match = row1
                        break

                    #ecart inf à 3%
                    if abs(surface1 - surface2) / max(surface1, surface2) < 0.03:
                        valeur_fonciere = pd.to_numeric(str(row1.get('Valeur fonciere', 0)).replace(',', '.'), errors='coerce')
                        if valeur_fonciere > best_value:
                            best_value = valeur_fonciere
                            best_match = row1

                if best_match is not None:
                    merged.append({**row2.to_dict(), **best_match.to_dict()})

    #df des résultats fusionnés
    df = pd.DataFrame(merged)
    return df

#application fonction
test4 = test_match4(vf, df)

Traitement du département : 13
Traitement du département : 92
Traitement du département : 69
Traitement du département : 03
Traitement du département : 56
Traitement du département : 75
Traitement du département : 29
Traitement du département : 91
Traitement du département : 62
Traitement du département : 21
Traitement du département : 11
Traitement du département : 02
Traitement du département : 59
Traitement du département : 94
Traitement du département : 44
Traitement du département : 50
Traitement du département : 37
Traitement du département : 81
Traitement du département : 67
Traitement du département : 93
Traitement du département : 83
Traitement du département : 63
Traitement du département : 95
Traitement du département : 76
Traitement du département : 33
Traitement du département : 78
Traitement du département : 38
Traitement du département : 51
Traitement du département : 57
Traitement du département : 77
Traitement du département : 35
Traitement du département : 54
Traiteme

In [28]:
print(len(test4))

73502


In [29]:
result3 = test4.drop_duplicates(subset=['Date_réception_DPE', 'Adresse_Normalisee', 'Surface_habitable_logement', 'Valeur fonciere', 'N°_étage_appartement', 'Etiquette_DPE', "Complément_d'adresse_logement", "Complément_d'adresse_bâtiment", "Date_établissement_DPE"])

In [30]:
print(len(result3))

69313


In [31]:
result3.to_csv("result3%.csv", index=False)

## TEST AUTRES ANNEES

Il semble que nous ne disposions que des données à partir de 2021, nous pouvons tout de même vérifier que c'est bien le cas car les fichiers valeurs foncières s'étendent jusqu'à 2014.

In [ ]:
df['Date_réception_DPE'] = pd.to_datetime(df['Date_réception_DPE'], errors='coerce')
df2014 = df[df['Date_réception_DPE'].dt.year == 2014].copy()

In [35]:
print(len(df2014))

0


In [ ]:
df2015 = df[df['Date_réception_DPE'].dt.year == 2015].copy()

In [37]:
print(len(df2015))

0


In [ ]:
df2016 = df[df['Date_réception_DPE'].dt.year == 2016].copy()

In [39]:
print(len(df2016))

0


In [ ]:
df2017 = df[df['Date_réception_DPE'].dt.year == 2017].copy()

In [41]:
print(len(df2017))

0


In [ ]:
df2018 = df[df['Date_réception_DPE'].dt.year == 2018].copy()

In [43]:
print(len(df2018))

0


In [ ]:
df2019 = df[df['Date_réception_DPE'].dt.year == 2019].copy()

In [45]:
print(len(df2019))

0


In [ ]:
df2020 = df[df['Date_réception_DPE'].dt.year == 2020].copy()

In [47]:
print(len(df2020))

0


In [ ]:
df2021 = df[df['Date_réception_DPE'].dt.year == 2021].copy()

In [49]:
print(len(df2021))

677703


In [ ]:
df2023= df[df['Date_réception_DPE'].dt.year == 2023].copy()

In [51]:
print(len(df2023))

1438813


Nous allons donc pouvoir seulement compiler les données pour les années : 2021, 2022,2023.

### ANALYSE DES DONNEES COMMUNES ENTRE DF ET DF2 (LOGEMENTS EXISTANTS ET NEUFS)

In [13]:
def trouver_doublons_colonnes_communes(df, df2):
    #colonnes communes
    colonnes_communes = list(set(df.columns).intersection(df2.columns))
    print(f"Colonnes communes : {colonnes_communes}")
    
    #doublons parmi colonnes communes
    doublons = pd.merge(df[colonnes_communes], df2[colonnes_communes], how='inner')
    return doublons

doublons = trouver_doublons_colonnes_communes(df, df2)

print("\nLignes en doublons pour les colonnes communes :")
print(doublons)

Colonnes communes : ['Version_DPE', 'Code_postal_(BAN)', 'Code_INSEE_(BAN)', 'Date_fin_validité_DPE', 'Identifiant__BAN', 'N°_région_(BAN)', 'Date_réception_DPE', 'Nom_résidence', "Complément_d'adresse_bâtiment", "Cage_d'escalier", 'Date_établissement_DPE', 'Nom__commune_(Brut)', 'N°_voie_(BAN)', 'Etiquette_DPE', 'N°_département_(BAN)', 'Adresse_brute', 'Nom__commune_(BAN)', 'Score_BAN', 'Code_postal_(brut)', 'Adresse_(BAN)', 'Etiquette_GES', 'Nom__rue_(BAN)', 'Coordonnée_cartographique_X_(BAN)', 'Modèle_DPE', 'Statut_géocodage', 'Type_bâtiment', 'Coordonnée_cartographique_Y_(BAN)', 'Adresse_Normalisee', 'Surface_habitable_logement', "Complément_d'adresse_logement", 'N°_étage_appartement']

Lignes en doublons pour les colonnes communes :
Empty DataFrame
Columns: [Version_DPE, Code_postal_(BAN), Code_INSEE_(BAN), Date_fin_validité_DPE, Identifiant__BAN, N°_région_(BAN), Date_réception_DPE, Nom_résidence, Complément_d'adresse_bâtiment, Cage_d'escalier, Date_établissement_DPE, Nom__commun

Il n'y a pas de doublons entre df et df2 donc on peut merge les deux séparément avec vf puis les compiler. 

### Merge de df2 avec vf pour 2022

In [14]:
df2['Date_réception_DPE'] = pd.to_datetime(df2['Date_réception_DPE'], errors='coerce')
df2 = df2[df2['Date_réception_DPE'].dt.year == 2022].copy()

In [18]:
vf = vf[vf['Code postal'].notna()]
df2 = df2[df2['N°_département_(BAN)'].notna()]

In [19]:
vf['Code postal'] = vf['Code postal'].astype(int).astype(str)
df2.loc[:, 'N°_département_(BAN)'] = df2['N°_département_(BAN)'].astype(str)

### 5%

In [22]:
dfneuf5 = test_match2(vf, df2)

Traitement du département : 29
Traitement du département : 69
Traitement du département : 33
Traitement du département : 83
Traitement du département : 31
Traitement du département : 13
Traitement du département : 93
Traitement du département : 92
Traitement du département : 14
Traitement du département : 81
Traitement du département : 07
Traitement du département : 30
Traitement du département : 84
Traitement du département : 85
Traitement du département : 34
Traitement du département : 63
Traitement du département : 77
Traitement du département : 66
Traitement du département : 51
Traitement du département : 57
Traitement du département : 44
Traitement du département : 91
Traitement du département : 47
Traitement du département : 64
Traitement du département : 87
Traitement du département : 88
Traitement du département : 50
Traitement du département : 38
Traitement du département : 72
Traitement du département : 74
Traitement du département : 79
Traitement du département : 35
Traiteme

In [23]:
print(len(dfneuf5))

2714


In [36]:
dfneuf5 = dfneuf5.drop_duplicates(subset=['Date_réception_DPE', 'Adresse_Normalisee', 'Surface_habitable_logement', 'Valeur fonciere', 'N°_étage_appartement', 'Etiquette_DPE', "Complément_d'adresse_logement", "Complément_d'adresse_bâtiment", "Date_établissement_DPE"])

In [37]:
print(len(dfneuf5))

1911


In [28]:
dfexistant5 = pd.read_csv("result5%.csv")

/var/folders/bf/ymcp1z4s7n77n5bhz3q_hgqc0000gn/T/ipykernel_1037/4222365088.py:1: DtypeWarning: Columns (15,58,60,62,65,67,75) have mixed types. Specify dtype option on import or set low_memory=False.
  dfexistant5 = pd.read_csv("result5%.csv")


### Compilation des deux 

In [66]:
# Compiler les deux DataFrames
def compiler_dataframes(df1, df2):
    #concatène les deux DataFrames
    df_compilé = pd.concat([df1, df2], ignore_index=True)
    #supprime doublons
    df_compilé = df_compilé.drop_duplicates()
    return df_compilé

In [ ]:
df_compilé = compiler_dataframes(dfexistant5, dfneuf5)

In [70]:
df_compilé.to_csv("result5%_2022_compilé.csv", index=False)

In [71]:
print(len(df_compilé))

79059


## 2021

In [41]:
vf2021 = pd.read_csv(
    "/Users/noelinecasteil/Documents/statapp/ValeursFoncieres/valeursfoncieres-2021.txt",
    sep="|",  
    encoding="utf-8",
    low_memory=False
)

In [42]:
# Convertir explicitement toutes les colonnes en chaînes
vf2021['Adresse'] = vf2021['No voie'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
                vf2021['Type de voie'].fillna('').astype(str) + " " + \
                vf2021['Voie'].fillna('').astype(str) + ", " + \
                vf2021['Code postal'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
                vf2021['Commune'].fillna('').astype(str)

vf2021['Adresse'] = vf2021['Adresse'].str.strip().replace(r'^\s*$', None, regex=True)  # Supprime les adresses vides

In [49]:
df['Date_réception_DPE'] = pd.to_datetime(df['Date_réception_DPE'], errors='coerce')
df2021 = df[df['Date_réception_DPE'].dt.year == 2021].copy()

In [50]:
vf2021 = vf2021[vf2021['Code postal'].notna()]
df2021 = df2021[df2021['N°_département_(BAN)'].notna()]

In [51]:
vf2021['Code postal'] = vf2021['Code postal'].astype(int).astype(str)
df2021.loc[:, 'N°_département_(BAN)'] = df2021['N°_département_(BAN)'].astype(str)

MATCHING 5%

In [53]:
dfexistant2021 = test_match2(vf2021, df2021)

Traitement du département : 25
Traitement du département : 44
Traitement du département : 36
Traitement du département : 29
Traitement du département : 43
Traitement du département : 06
Traitement du département : 88
Traitement du département : 78
Traitement du département : 14
Traitement du département : 80
Traitement du département : 54
Traitement du département : 82
Traitement du département : 49
Traitement du département : 86
Traitement du département : 52
Traitement du département : 64
Traitement du département : 59
Traitement du département : 77
Traitement du département : 72
Traitement du département : 67
Traitement du département : 60
Traitement du département : 89
Traitement du département : 41
Traitement du département : 30
Traitement du département : 69
Traitement du département : 81
Traitement du département : 85
Traitement du département : 57
Traitement du département : 2B
Traitement du département : 76
Traitement du département : 35
Traitement du département : 56
Traiteme

In [55]:
dfexistant2021 = dfexistant2021.drop_duplicates(subset=['Date_réception_DPE', 'Adresse_Normalisee', 'Surface_habitable_logement', 'Valeur fonciere', 'N°_étage_appartement', 'Etiquette_DPE', "Complément_d'adresse_logement", "Complément_d'adresse_bâtiment", "Date_établissement_DPE"])

In [56]:
print(len(dfexistant2021))

21335


LOGEMENTS NEUFS

In [58]:
df2['Date_réception_DPE'] = pd.to_datetime(df2['Date_réception_DPE'], errors='coerce')
df22021 = df2[df2['Date_réception_DPE'].dt.year == 2021].copy()

In [60]:
df22021 = df22021[df22021['N°_département_(BAN)'].notna()]

In [61]:
df22021.loc[:, 'N°_département_(BAN)'] = df22021['N°_département_(BAN)'].astype(str)

DF NEUF à 5% 

In [62]:
dfneuf2021 = test_match2(vf2021, df22021)

Traitement du département : 33
Traitement du département : 91
Traitement du département : 64
Traitement du département : 57
Traitement du département : 63
Traitement du département : 80
Traitement du département : 74
Traitement du département : 95
Traitement du département : 26
Traitement du département : 37
Traitement du département : 77
Traitement du département : 76
Traitement du département : 38
Traitement du département : 35
Traitement du département : 92
Traitement du département : 56
Traitement du département : 31
Traitement du département : 94
Traitement du département : 89
Traitement du département : 69
Traitement du département : 01
Traitement du département : 42
Traitement du département : 44
Traitement du département : 93
Traitement du département : 21
Traitement du département : 45
Traitement du département : 17
Traitement du département : 68
Traitement du département : 58
Traitement du département : 73
Traitement du département : 13
Traitement du département : 14
Traiteme

In [64]:
dfneuf2021 = dfneuf2021.drop_duplicates(subset=['Date_réception_DPE', 'Adresse_Normalisee', 'Surface_habitable_logement', 'Valeur fonciere', 'N°_étage_appartement', 'Etiquette_DPE', "Complément_d'adresse_logement", "Complément_d'adresse_bâtiment", "Date_établissement_DPE"])

In [65]:
print(len(dfneuf2021))

676


In [67]:
df2021compile = compiler_dataframes(dfexistant2021, dfneuf2021)

In [69]:
df2021compile.to_csv("result5%_2021_compilé.csv", index=False)

### 2023

On n'a pas les valeurs foncières de 2023 mince

# DATA FRAME FINAL POUR 5%

In [73]:
df1 = pd.read_csv("result5%_2021_compilé.csv")
df2 = pd.read_csv("result5%_2022_compilé.csv")

/var/folders/bf/ymcp1z4s7n77n5bhz3q_hgqc0000gn/T/ipykernel_1037/1554524822.py:1: DtypeWarning: Columns (58,60,75) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("result5%_2021_compilé.csv")
/var/folders/bf/ymcp1z4s7n77n5bhz3q_hgqc0000gn/T/ipykernel_1037/1554524822.py:2: DtypeWarning: Columns (15,58,60,62,65,67,75) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("result5%_2022_compilé.csv")


In [74]:
dffinal5 = compiler_dataframes(df1, df2)

In [76]:
dffinal5.to_csv("dataframefinal_5%.csv", index=False)